In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
from matplotlib import patches
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import os
import math
import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.metrics import roc_curve, roc_auc_score, auc

import scipy.stats as st
from scipy.stats import poisson
from scipy.interpolate import interp1d
from scipy.stats import norm


from sklearn.neighbors import KernelDensity 
from sklearn.model_selection import GridSearchCV

import scipy.integrate as integrate


import xgboost
from xgboost import XGBRegressor, XGBClassifier

In [2]:
import matplotlib as mpl
mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12) 

# Reading the data

In [3]:
Wgamma_fidcross  = 584.6234734948906
Wjets_fidcross   = 606.8765059826087
ttgamma_fidcross = 28.773979008
B_fidcross_tot   = Wgamma_fidcross + Wjets_fidcross + ttgamma_fidcross
B_fidcross_tot

1220.2739584854994

In [4]:
B_expected = (2877.3979008 + 58462.347349489064 + 60687.65059826087)

In [5]:
##############
# input data #
##############

# photon -> (pT, eta, phi) from the leading photon
# jet -> (pT, eta, phi) from the leading jet
# lepton -> (pT, eta, phi) from the lepton (electron or muon)
# MET -> (pT, eta, phi) eta = 0 by construction, but phi is important because we have a lot of jets that we do not use
# hadinfo -> (HT, num_jets)  HT: sum PT of all the jets, number of jets (b-tag or not)


######
# SM #
######

# W gamma + W gamma jet #
# Dominant # Np es el W jet el dominant??

datFolderWgamma = '../data/Wgamma/'

photon_Wgamma  = np.loadtxt(datFolderWgamma + 'photon_3.dat', delimiter=' ')
jet_Wgamma     = np.loadtxt(datFolderWgamma + 'jet_3.dat', delimiter=' ')
lepton_Wgamma  = np.loadtxt(datFolderWgamma + 'lepton_3.dat', delimiter=' ')
MET_Wgamma     = np.loadtxt(datFolderWgamma + 'MET_3.dat', delimiter=' ')
hadinfo_Wgamma = np.loadtxt(datFolderWgamma + 'hadinfo_3.dat', delimiter=' ')


# W jet + W jet jet #

datFolderWjets = '../data/Wjets/'

photon_Wjets  = np.loadtxt(datFolderWjets + 'photon_3.dat', delimiter=' ')
jet_Wjets     = np.loadtxt(datFolderWjets + 'jet_3.dat', delimiter=' ')
lepton_Wjets  = np.loadtxt(datFolderWjets + 'lepton_3.dat', delimiter=' ')
MET_Wjets     = np.loadtxt(datFolderWjets + 'MET_3.dat', delimiter=' ')
hadinfo_Wjets = np.loadtxt(datFolderWjets + 'hadinfo_3.dat', delimiter=' ')


# tt gamma + tt gamma jet #

datFolderttgamma = '../data/ttGamma/'
 
photon_ttgamma  = np.loadtxt(datFolderttgamma + 'photon_3.dat', delimiter=' ')
jet_ttgamma     = np.loadtxt(datFolderttgamma + 'jet_3.dat', delimiter=' ')
lepton_ttgamma  = np.loadtxt(datFolderttgamma + 'lepton_3.dat', delimiter=' ')
MET_ttgamma     = np.loadtxt(datFolderttgamma + 'MET_3.dat', delimiter=' ')
hadinfo_ttgamma = np.loadtxt(datFolderttgamma + 'hadinfo_3.dat', delimiter=' ')


photon_SM  = np.concatenate([photon_Wgamma, photon_Wjets, photon_ttgamma])
jet_SM     = np.concatenate([jet_Wgamma, jet_Wjets, jet_ttgamma]) 
lepton_SM  = np.concatenate([lepton_Wgamma, lepton_Wjets, lepton_ttgamma])
MET_SM     = np.concatenate([MET_Wgamma, MET_Wjets, MET_ttgamma])
hadinfo_SM = np.concatenate([hadinfo_Wgamma, hadinfo_Wjets, hadinfo_ttgamma])


In [6]:
Wgamma_sample = len(photon_Wgamma)
Wjets_sample = len(photon_Wjets)
ttgamma_sample = len(photon_ttgamma)

In [7]:
bkg_flag = np.zeros(len(photon_SM))

bkg_flag[:Wgamma_sample] = 1
bkg_flag[Wgamma_sample:(Wgamma_sample + Wjets_sample)] = 2
bkg_flag[(Wgamma_sample + Wjets_sample):len(photon_SM)] = 3

In [8]:
######
# NP #
######

datFolderNP = '../data/Signal/'

photon_NP  = []
jet_NP     = []
lepton_NP  = []
MET_NP     = []
hadinfo_NP = []
name       = []
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    name.append('BP' + str(i))
    photon_NP.append( np.loadtxt(datFolderNP + 'BP' + str(i) + '_photon_3.dat', delimiter=' ') )
    jet_NP.append( np.loadtxt(datFolderNP + 'BP' + str(i) + '_jet_3.dat', delimiter=' ') )
    lepton_NP.append( np.loadtxt(datFolderNP + 'BP' + str(i) + '_lepton_3.dat', delimiter=' ') )
    MET_NP.append( np.loadtxt(datFolderNP + 'BP' + str(i) + '_MET_3.dat', delimiter=' ') )
    hadinfo_NP.append( np.loadtxt(datFolderNP + 'BP' + str(i) + '_hadinfo_3.dat', delimiter=' ') )

In [9]:
print('SM', jet_SM.shape)
for i in range(len(jet_NP)):
    print(name[i], jet_NP[i].shape)

SM (5892229, 3)
BP1 (238160, 3)
BP2 (246196, 3)
BP3 (242249, 3)
BP4 (301575, 3)
BP5 (266047, 3)
BP6 (203155, 3)
BP7 (315960, 3)
BP8 (280065, 3)
BP9 (225958, 3)
BP10 (326687, 3)
BP11 (292051, 3)
BP12 (244271, 3)
BP13 (334782, 3)
BP14 (301215, 3)
BP15 (258277, 3)


In [10]:
cross_section_bps    = np.asarray([190, 189, 189, 92, 92, 92, 49, 48, 48, 27, 27, 27, 16, 16, 16])
branching_ratios_bps = np.asarray([0.14, 0.37, 0.73, 0.15, 0.36, 0.67, 0.16, 0.36, 0.62, 0.17, 0.35, 0.58, 0.16, 0.32, 0.52])

signal_cross        = cross_section_bps * branching_ratios_bps *  0.2134 # fb
signal_init_sim_eve = np.asarray([836673,1e6,1.4e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6])
signal_cut_ev       = np.asarray([238160,246196,24249,301575,266047,203155,315960,280065,225958,326687,292051,244271,334782,301215,258277])
signal_NLO          = 1.25

cross       = signal_cross * signal_NLO
aceptancia  = signal_cut_ev / (1. * signal_init_sim_eve)
luminosidad = 100

signal_fidcross = cross * aceptancia
S_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', S_expected)

fiducial cross section:  [2.01975705 4.59249772 0.63746286 1.11014281 2.35046139 3.34036999
 0.66077347 1.29094281 1.79376306 0.39998985 0.73619851 1.02039447
 0.22861593 0.4113874  0.57320964]
Events expected:  [201.97570473 459.24977152  63.74628649 111.01428112 235.04613937
 334.03699928  66.07734672 129.09428136 179.37630638  39.99898458
  73.61985102 102.03944697  22.86159322  41.13873984  57.32096427]


In [11]:
# other variables, individual channels:

HT_main_Wgamma = photon_Wgamma[:,0] + lepton_Wgamma[:,0] + jet_Wgamma[:,0]

MET_sig_Wgamma = np.divide( MET_Wgamma[:,0],np.sqrt(HT_main_Wgamma) )

m_eff_Wgamma = MET_Wgamma[:,0] + photon_Wgamma[:,0] + lepton_Wgamma[:,0] + jet_Wgamma[:,0]

phi_MET_photon_Wgamma = np.absolute( MET_Wgamma[:,2] - photon_Wgamma[:,2] )
phi_MET_jet_Wgamma    = np.absolute( MET_Wgamma[:,2] - jet_Wgamma[:,2] )
phi_MET_lepton_Wgamma = np.absolute( MET_Wgamma[:,2] - lepton_Wgamma[:,2] )

mT_photon_Wgamma = np.sqrt( (2*photon_Wgamma[:,0]*MET_Wgamma[:,0]) * (1 - np.cos(phi_MET_photon_Wgamma)) )
mT_jet_Wgamma    = np.sqrt( (2*jet_Wgamma[:,0]*MET_Wgamma[:,0]) * (1 - np.cos(phi_MET_jet_Wgamma)) )
mT_lepton_Wgamma = np.sqrt( (2*lepton_Wgamma[:,0]*MET_Wgamma[:,0]) * (1 - np.cos(phi_MET_lepton_Wgamma)) )

phi_photon_jet_Wgamma    = np.absolute( photon_Wgamma[:,2] - jet_Wgamma[:,2] )
phi_photon_lepton_Wgamma = np.absolute( photon_Wgamma[:,2] - lepton_Wgamma[:,2] )
phi_lepton_jet_Wgamma    = np.absolute( lepton_Wgamma[:,2] - jet_Wgamma[:,2] )

phi_photon_jet_Wgamma[phi_photon_jet_Wgamma>np.pi]       = (2.* np.pi) - phi_photon_jet_Wgamma[phi_photon_jet_Wgamma>np.pi]
phi_photon_lepton_Wgamma[phi_photon_lepton_Wgamma>np.pi] = (2.* np.pi) - phi_photon_lepton_Wgamma[phi_photon_lepton_Wgamma>np.pi]
phi_lepton_jet_Wgamma[phi_lepton_jet_Wgamma>np.pi]       = (2.* np.pi) - phi_lepton_jet_Wgamma[phi_lepton_jet_Wgamma>np.pi]

deltaR_photon_jet_Wgamma    = np.sqrt( (photon_Wgamma[:,1]-jet_Wgamma[:,1])**2 + (phi_photon_jet_Wgamma)**2 )
deltaR_photon_lepton_Wgamma = np.sqrt( (photon_Wgamma[:,1]-lepton_Wgamma[:,1])**2 + (phi_photon_lepton_Wgamma)**2 )
deltaR_lepton_jet_Wgamma    = np.sqrt( (lepton_Wgamma[:,1]-jet_Wgamma[:,1])**2 + (phi_lepton_jet_Wgamma)**2 )

HT_main_Wjets = photon_Wjets[:,0]+lepton_Wjets[:,0]+jet_Wjets[:,0]

MET_sig_Wjets = np.divide( MET_Wjets[:,0],np.sqrt(HT_main_Wjets) )

m_eff_Wjets = MET_Wjets[:,0] + photon_Wjets[:,0]+lepton_Wjets[:,0]+jet_Wjets[:,0]

phi_MET_photon_Wjets = np.absolute( MET_Wjets[:,2] - photon_Wjets[:,2] )
phi_MET_jet_Wjets    = np.absolute( MET_Wjets[:,2] - jet_Wjets[:,2] )
phi_MET_lepton_Wjets = np.absolute( MET_Wjets[:,2] - lepton_Wjets[:,2] )

mT_photon_Wjets = np.sqrt( (2*photon_Wjets[:,0]*MET_Wjets[:,0]) * (1 - np.cos(phi_MET_photon_Wjets)) )
mT_jet_Wjets    = np.sqrt( (2*jet_Wjets[:,0]*MET_Wjets[:,0]) * (1 - np.cos(phi_MET_jet_Wjets)) )
mT_lepton_Wjets = np.sqrt( (2*lepton_Wjets[:,0]*MET_Wjets[:,0]) * (1 - np.cos(phi_MET_lepton_Wjets)) )

phi_photon_jet_Wjets    = np.absolute( photon_Wjets[:,2] - jet_Wjets[:,2] )
phi_photon_lepton_Wjets = np.absolute( photon_Wjets[:,2] - lepton_Wjets[:,2] )
phi_lepton_jet_Wjets    = np.absolute( lepton_Wjets[:,2] - jet_Wjets[:,2] )

phi_photon_jet_Wjets[phi_photon_jet_Wjets>np.pi]       = (2.* np.pi) - phi_photon_jet_Wjets[phi_photon_jet_Wjets>np.pi]
phi_photon_lepton_Wjets[phi_photon_lepton_Wjets>np.pi] = (2.* np.pi) - phi_photon_lepton_Wjets[phi_photon_lepton_Wjets>np.pi]
phi_lepton_jet_Wjets[phi_lepton_jet_Wjets>np.pi]       = (2.* np.pi) - phi_lepton_jet_Wjets[phi_lepton_jet_Wjets>np.pi]

deltaR_photon_jet_Wjets    = np.sqrt( (photon_Wjets[:,1]-jet_Wjets[:,1])**2 + (phi_photon_jet_Wjets)**2 )
deltaR_photon_lepton_Wjets = np.sqrt( (photon_Wjets[:,1]-lepton_Wjets[:,1])**2 + (phi_photon_lepton_Wjets)**2 )
deltaR_lepton_jet_Wjets    = np.sqrt( (lepton_Wjets[:,1]-jet_Wjets[:,1])**2 + (phi_lepton_jet_Wjets)**2 )


HT_main_ttgamma = photon_ttgamma[:,0]+lepton_ttgamma[:,0]+jet_ttgamma[:,0]

MET_sig_ttgamma = np.divide( MET_ttgamma[:,0],np.sqrt(HT_main_ttgamma) )

m_eff_ttgamma = MET_ttgamma[:,0] + photon_ttgamma[:,0]+lepton_ttgamma[:,0]+jet_ttgamma[:,0]

phi_MET_photon_ttgamma = np.absolute( MET_ttgamma[:,2] - photon_ttgamma[:,2] )
phi_MET_jet_ttgamma    = np.absolute( MET_ttgamma[:,2] - jet_ttgamma[:,2] )
phi_MET_lepton_ttgamma = np.absolute( MET_ttgamma[:,2] - lepton_ttgamma[:,2] )

mT_photon_ttgamma = np.sqrt( (2*photon_ttgamma[:,0]*MET_ttgamma[:,0]) * (1 - np.cos(phi_MET_photon_ttgamma)) )
mT_jet_ttgamma    = np.sqrt( (2*jet_ttgamma[:,0]*MET_ttgamma[:,0]) * (1 - np.cos(phi_MET_jet_ttgamma)) )
mT_lepton_ttgamma = np.sqrt( (2*lepton_ttgamma[:,0]*MET_ttgamma[:,0]) * (1 - np.cos(phi_MET_lepton_ttgamma)) )

phi_photon_jet_ttgamma    = np.absolute( photon_ttgamma[:,2] - jet_ttgamma[:,2] )
phi_photon_lepton_ttgamma = np.absolute( photon_ttgamma[:,2] - lepton_ttgamma[:,2] )
phi_lepton_jet_ttgamma    = np.absolute( lepton_ttgamma[:,2] - jet_ttgamma[:,2] )

phi_photon_jet_ttgamma[phi_photon_jet_ttgamma>np.pi]       = (2.* np.pi) - phi_photon_jet_ttgamma[phi_photon_jet_ttgamma>np.pi]
phi_photon_lepton_ttgamma[phi_photon_lepton_ttgamma>np.pi] = (2.* np.pi) - phi_photon_lepton_ttgamma[phi_photon_lepton_ttgamma>np.pi]
phi_lepton_jet_ttgamma[phi_lepton_jet_ttgamma>np.pi]       = (2.* np.pi) - phi_lepton_jet_ttgamma[phi_lepton_jet_ttgamma>np.pi]

deltaR_photon_jet_ttgamma    = np.sqrt( (photon_ttgamma[:,1]-jet_ttgamma[:,1])**2 + (phi_photon_jet_ttgamma)**2 )
deltaR_photon_lepton_ttgamma = np.sqrt( (photon_ttgamma[:,1]-lepton_ttgamma[:,1])**2 + (phi_photon_lepton_ttgamma)**2 )
deltaR_lepton_jet_ttgamma    = np.sqrt( (lepton_ttgamma[:,1]-jet_ttgamma[:,1])**2 + (phi_lepton_jet_ttgamma)**2 )



# other variables

HT_main_SM = photon_SM[:,0]+lepton_SM[:,0]+jet_SM[:,0]

MET_sig_SM = np.divide( MET_SM[:,0],np.sqrt(HT_main_SM) )

m_eff_SM = MET_SM[:,0] + photon_SM[:,0]+lepton_SM[:,0]+jet_SM[:,0]

phi_MET_photon_SM = np.absolute( MET_SM[:,2] - photon_SM[:,2] )
phi_MET_jet_SM    = np.absolute( MET_SM[:,2] - jet_SM[:,2] )
phi_MET_lepton_SM = np.absolute( MET_SM[:,2] - lepton_SM[:,2] )

mT_photon_SM = np.sqrt( (2*photon_SM[:,0]*MET_SM[:,0]) * (1 - np.cos(phi_MET_photon_SM)) )
mT_jet_SM    = np.sqrt( (2*jet_SM[:,0]*MET_SM[:,0]) * (1 - np.cos(phi_MET_jet_SM)) )
mT_lepton_SM = np.sqrt( (2*lepton_SM[:,0]*MET_SM[:,0]) * (1 - np.cos(phi_MET_lepton_SM)) )

phi_photon_jet_SM    = np.absolute( photon_SM[:,2] - jet_SM[:,2] )
phi_photon_lepton_SM = np.absolute( photon_SM[:,2] - lepton_SM[:,2] )
phi_lepton_jet_SM    = np.absolute( lepton_SM[:,2] - jet_SM[:,2] )

phi_photon_jet_SM[phi_photon_jet_SM>np.pi]       = (2.* np.pi) - phi_photon_jet_SM[phi_photon_jet_SM>np.pi]
phi_photon_lepton_SM[phi_photon_lepton_SM>np.pi] = (2.* np.pi) - phi_photon_lepton_SM[phi_photon_lepton_SM>np.pi]
phi_lepton_jet_SM[phi_lepton_jet_SM>np.pi]       = (2.* np.pi) - phi_lepton_jet_SM[phi_lepton_jet_SM>np.pi]

deltaR_photon_jet_SM    = np.sqrt( (photon_SM[:,1]-jet_SM[:,1])**2 + (phi_photon_jet_SM)**2 )
deltaR_photon_lepton_SM = np.sqrt( (photon_SM[:,1]-lepton_SM[:,1])**2 + (phi_photon_lepton_SM)**2 )
deltaR_lepton_jet_SM    = np.sqrt( (lepton_SM[:,1]-jet_SM[:,1])**2 + (phi_lepton_jet_SM)**2 )

In [12]:
# other variables NEW PHYSICS
HT_main_NP = []
MET_sig_NP = []
m_eff_NP   = []

phi_MET_photon_NP = []
phi_MET_jet_NP    = []
phi_MET_lepton_NP = []

mT_photon_NP = []
mT_jet_NP    = []
mT_lepton_NP = []

phi_photon_jet_NP    = []
phi_photon_lepton_NP = []
phi_lepton_jet_NP    = []

phi_photon_jet_NP    = []
phi_photon_lepton_NP = []
phi_lepton_jet_NP    = []

deltaR_photon_jet_NP    = []
deltaR_photon_lepton_NP = []
deltaR_lepton_jet_NP    = []

for i in range(len(photon_NP)):
    HT_main_NP.append( photon_NP[i][:,0] + lepton_NP[i][:,0] + jet_NP[i][:,0] )

    MET_sig_NP.append( np.divide( MET_NP[i][:,0],np.sqrt(HT_main_NP[i]) ) )

    m_eff_NP.append( MET_NP[i][:,0] + photon_NP[i][:,0] + lepton_NP[i][:,0] + jet_NP[i][:,0] )

    phi_MET_photon_NP.append( np.absolute( MET_NP[i][:,2] - photon_NP[i][:,2] ) )
    phi_MET_jet_NP.append( np.absolute( MET_NP[i][:,2] - jet_NP[i][:,2] ) )
    phi_MET_lepton_NP.append( np.absolute( MET_NP[i][:,2] - lepton_NP[i][:,2] ) )

    mT_photon_NP.append( np.sqrt( (2*photon_NP[i][:,0]*MET_NP[i][:,0]) * (1 - np.cos(phi_MET_photon_NP[i])) ) )
    mT_jet_NP.append( np.sqrt( (2*jet_NP[i][:,0]*MET_NP[i][:,0]) * (1 - np.cos(phi_MET_jet_NP[i])) ) )
    mT_lepton_NP.append( np.sqrt( (2*lepton_NP[i][:,0]*MET_NP[i][:,0]) * (1 - np.cos(phi_MET_lepton_NP[i])) ) )

    phi_photon_jet_NP.append( np.absolute( photon_NP[i][:,2] - jet_NP[i][:,2] ) )
    phi_photon_lepton_NP.append( np.absolute( photon_NP[i][:,2] - lepton_NP[i][:,2] ) )
    phi_lepton_jet_NP.append( np.absolute( lepton_NP[i][:,2] - jet_NP[i][:,2] ) )

    phi_photon_jet_NP[i][phi_photon_jet_NP[i] > np.pi]       = (2.* np.pi) - phi_photon_jet_NP[i][phi_photon_jet_NP[i] > np.pi]
    phi_photon_lepton_NP[i][phi_photon_lepton_NP[i] > np.pi] = (2.* np.pi) - phi_photon_lepton_NP[i][phi_photon_lepton_NP[i] > np.pi]
    phi_lepton_jet_NP[i][phi_lepton_jet_NP[i] > np.pi]       = (2.* np.pi) - phi_lepton_jet_NP[i][phi_lepton_jet_NP[i] > np.pi]

    deltaR_photon_jet_NP.append( np.sqrt( (photon_NP[i][:,1]-jet_NP[i][:,1])**2 + (phi_photon_jet_NP[i])**2 ) )
    deltaR_photon_lepton_NP.append( np.sqrt( (photon_NP[i][:,1]-lepton_NP[i][:,1])**2 + (phi_photon_lepton_NP[i])**2 ) )
    deltaR_lepton_jet_NP.append( np.sqrt( (lepton_NP[i][:,1]-jet_NP[i][:,1])**2 + (phi_lepton_jet_NP[i])**2 ) )

In [13]:
print(photon_SM.shape)
print(photon_NP[0].shape)

(5892229, 4)
(238160, 4)


# Analysis

In [14]:
# LOW LEVEL VARIABLES ----------------------
# photon_SM[:,0] : PT
# photon_SM[:,1] : eta
# photon_SM[:,2] : phi

print(photon_SM.shape)
print(jet_SM.shape)
print(lepton_SM.shape)
print(MET_SM.shape)
#-------------------------------------------

# HIGH LEVEL VARIABLES ----------------------
# hadinfo_SM[:,0] : hadinfo
# hadinfo_SM[:,1] : number jets
# HT_main_SM : had info total
# MET_sig_SM : MET significace
# m_eff_SM : Effective mass
# mT_photon_SM : transverse mass
print(hadinfo_SM.shape)
print(HT_main_SM.shape)
print(MET_sig_SM.shape)
print(m_eff_SM.shape)
print(mT_photon_SM.shape)
print(mT_jet_SM.shape)
print(mT_lepton_SM.shape)
#-------------------------------------------

(5730945, 4)
(5730945, 3)
(5730945, 4)
(5730945, 3)
(5730945, 2)
(5730945,)
(5730945,)
(5730945,)
(5730945,)
(5730945,)
(5730945,)


In [15]:
# bp correspond to benchmark points (bp + 1) !!!!
bp = 0

# LOW LEVEL VARIABLES ----------------------
# photon_NP[bp][:,0] : PT
# photon_NP[bp][:,1] : eta
# photon_NP[bp][:,2] : phi

print(photon_NP[bp].shape)
print(jet_NP[bp].shape)
print(lepton_NP[bp].shape)
print(MET_NP[bp].shape)
#-------------------------------------------

# HIGH LEVEL VARIABLES ----------------------
# hadinfo_NP[bp][:,0] : hadinfo
# hadinfo_NP[bp][:,1] : number jets
# HT_main_NP[bp] : had info total
# MET_sig_NP[bp] : MET significace
# m_eff_NP[bp] : Effective mass
# mT_photon_NP[bp] : transverse mass
print(hadinfo_NP[bp].shape)
print(HT_main_NP[bp].shape)
print(MET_sig_NP[bp].shape)
print(m_eff_NP[bp].shape)
print(mT_photon_NP[bp].shape)
print(mT_jet_NP[bp].shape)
print(mT_lepton_NP[bp].shape)
#-------------------------------------------

(238160, 4)
(238160, 3)
(238160, 4)
(238160, 3)
(238160, 2)
(238160,)
(238160,)
(238160,)
(238160,)
(238160,)
(238160,)


In [18]:
df = pd.DataFrame()

lst = ['photon $p_T$ [GeV]', 'photon $\eta$', 'photon $\phi$', 'num photon',
       'jet $p_T$ [GeV]', 'jet $\eta$', 'jet $\phi$',
       'lepton $p_T$ [GeV]', 'lepton $\eta$', 'lepton $\phi$', 'num lepton',
       'MET $p_T$ [GeV]', 'MET $\phi$',
       '$H_T$ (had) [GeV]', 'num_jets',
       '$H_T$ (main) [GeV]', 'MET sig [GeV]', '$m_{eff}$ [GeV]',
       '$m_T$ photon [GeV]', '$m_T$ jet [GeV]', '$m_T$ lepton [GeV]',
       '$\Delta \phi$ (MET-photon)', '$\Delta \phi$ (MET-jet)', '$\Delta \phi$ (MET-lepton)', 
       '$\Delta R$ (photon-jet)', '$\Delta R$ (photon-lepton)', '$\Delta R$ (lepton-jet)', 
       'SM=0 - NP=1']

df[lst[0]]= np.concatenate([photon_SM[:,0],photon_NP[bp][:,0]])
df[lst[1]]= np.concatenate([photon_SM[:,1],photon_NP[bp][:,1]])
df[lst[2]]= np.concatenate([photon_SM[:,2],photon_NP[bp][:,2]])
df[lst[3]]= np.concatenate([photon_SM[:,3],photon_NP[bp][:,3]])

df[lst[4]]= np.concatenate([jet_SM[:,0],jet_NP[bp][:,0]])
df[lst[5]]= np.concatenate([jet_SM[:,1],jet_NP[bp][:,1]])
df[lst[6]]= np.concatenate([jet_SM[:,2],jet_NP[bp][:,2]])

df[lst[7]]= np.concatenate([lepton_SM[:,0],lepton_NP[bp][:,0]])
df[lst[8]]= np.concatenate([lepton_SM[:,1],lepton_NP[bp][:,1]])
df[lst[9]]= np.concatenate([lepton_SM[:,2],lepton_NP[bp][:,2]])
df[lst[10]]= np.concatenate([lepton_SM[:,3],lepton_NP[bp][:,3]])

df[lst[11]]= np.concatenate([MET_SM[:,0],MET_NP[bp][:,0]])
df[lst[12]]= np.concatenate([MET_SM[:,2],MET_NP[bp][:,2]])

df[lst[13]]= np.concatenate([hadinfo_SM[:,0],hadinfo_NP[bp][:,0]])
df[lst[14]]= np.concatenate([hadinfo_SM[:,1],hadinfo_NP[bp][:,1]])

df[lst[15]]= np.concatenate([HT_main_SM,HT_main_NP[bp][:]])
df[lst[16]]= np.concatenate([MET_sig_SM,MET_sig_NP[bp][:]])
df[lst[17]]= np.concatenate([m_eff_SM,m_eff_NP[bp][:]])

df[lst[18]]= np.concatenate([mT_photon_SM,mT_photon_NP[bp][:]])
df[lst[19]]= np.concatenate([mT_jet_SM,mT_jet_NP[bp][:]])
df[lst[20]]= np.concatenate([mT_lepton_SM,mT_lepton_NP[bp][:]])

df[lst[21]]= np.concatenate([phi_MET_photon_SM,phi_MET_photon_NP[bp][:]])
df[lst[22]]= np.concatenate([phi_MET_jet_SM,phi_MET_jet_NP[bp][:]])
df[lst[23]]= np.concatenate([phi_MET_lepton_SM,phi_MET_lepton_NP[bp][:]])

df[lst[24]]= np.concatenate([deltaR_photon_jet_SM,deltaR_photon_jet_NP[bp][:]])
df[lst[25]]= np.concatenate([deltaR_photon_lepton_SM,deltaR_photon_lepton_NP[bp][:]])
df[lst[26]]= np.concatenate([deltaR_lepton_jet_SM,deltaR_lepton_jet_NP[bp][:]])


df['bkg_flag']= np.concatenate([np.zeros(len(photon_NP[bp])),bkg_flag])

df[lst[27]]= np.concatenate([np.zeros(len(photon_SM)),np.ones(len(photon_NP[bp][:]))])

df_array = np.asarray(df)
df

,photon $p_T$ [GeV],photon $\eta$,photon $\phi$,num photon,jet $p_T$ [GeV],jet $\eta$,jet $\phi$,lepton $p_T$ [GeV],lepton $\eta$,lepton $\phi$,...,$m_T$ jet [GeV],$m_T$ lepton [GeV],$\Delta \phi$ (MET-photon),$\Delta \phi$ (MET-jet),$\Delta \phi$ (MET-lepton),$\Delta R$ (photon-jet),$\Delta R$ (photon-lepton),$\Delta R$ (lepton-jet),bkg_flag,SM=0 - NP=1
0,12.30,2.102,-2.640,1.0,277.40,0.309,-0.144,62.97,1.864,-3.091,...,508.208040,23.774707,0.258,2.754,0.193,3.073250,0.509946,3.332092,0.0,0.0
1,19.65,-0.034,2.512,1.0,284.92,-0.875,-0.986,151.82,-0.509,1.995,...,354.243591,72.508431,0.040,3.538,0.557,2.909388,0.702078,3.003384,0.0,0.0
2,14.88,1.119,-1.349,1.0,176.83,2.457,-0.259,37.05,0.929,-2.610,...,338.439394,73.104899,4.044,2.954,5.305,1.725788,1.275234,2.803923,0.0,0.0
3,62.71,-0.144,2.679,1.0,157.32,0.248,0.038,22.12,1.172,1.521,...,89.822819,47.418325,2.031,0.610,0.873,2.669934,1.752946,1.747302,0.0,0.0
4,24.21,1.212,1.649,1.0,204.53,0.983,-1.499,86.94,0.914,2.213,...,310.848302,68.078092,0.120,3.028,0.684,3.143537,0.637887,2.572111,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5969100,45.67,0.262,-2.240,1.0,303.61,-1.577,1.250,25.54,-0.577,-0.883,...,529.762825,81.136708,0.246,3.244,1.111,3.344220,1.595422,2.355778,3.0,1.0
5969101,33.13,0.660,1.200,1.0,218.63,-1.530,-0.333,15.07,1.772,-2.669,...,345.330583,34.188115,1.641,3.174,5.510,2.673236,2.657976,4.044762,3.0,1.0
5969102,76.92,0.187,-1.864,1.0,227.64,-0.029,1.323,17.11,0.778,-0.086,...,389.024090,84.630336,0.052,3.239,1.830,3.103711,1.873650,1.623740,3.0,1.0
5969103,89.06,1.678,-1.271,1.0,193.55,0.503,2.000,17.80,2.123,1.301,...,349.474324,99.817258,0.116,3.155,2.456,3.233247,2.610212,1.764370,3.0,1.0


In [19]:
df.columns

Index(['photon $p_T$ [GeV]', 'photon $\eta$', 'photon $\phi$', 'num photon',
       'jet $p_T$ [GeV]', 'jet $\eta$', 'jet $\phi$', 'lepton $p_T$ [GeV]',
       'lepton $\eta$', 'lepton $\phi$', 'num lepton', 'MET $p_T$ [GeV]',
       'MET $\phi$', '$H_T$ (had) [GeV]', 'num_jets', '$H_T$ (main) [GeV]',
       'MET sig [GeV]', '$m_{eff}$ [GeV]', '$m_T$ photon [GeV]',
       '$m_T$ jet [GeV]', '$m_T$ lepton [GeV]', '$\Delta \phi$ (MET-photon)',
       '$\Delta \phi$ (MET-jet)', '$\Delta \phi$ (MET-lepton)',
       '$\Delta R$ (photon-jet)', '$\Delta R$ (photon-lepton)',
       '$\Delta R$ (lepton-jet)', 'bkg_flag', 'SM=0 - NP=1'],
      dtype='object')

In [20]:
df.columns[21]

'$\\Delta \\phi$ (MET-photon)'

In [21]:
indices_cut = np.where( 
                        #(df_array[:,0] <= 200) & # photon $p_T$ [GeV]
                        #(df_array[:,1] <= 200) & # photon $\eta$ 
                        #(df_array[:,2] <= 200) & # photon $\phi$
                        #(df_array[:,3] <= 200) & # num photons
                        #(df_array[:,4] <= 200) & # jet $p_T$ [GeV] 
                        #(df_array[:,5] <= 200) & # jet $\eta$
                        #(df_array[:,6] <= 200) & # jet $\phi$
                        (df_array[:,7] <= 200) & # lepton $p_T$ [GeV] 
                        #(df_array[:,8] <= 200) & # lepton $\eta$
                        #(df_array[:,9] <= 200) & # lepton $\phi$
                        #(df_array[:,10] <= 200) & # num leptons
                        #(df_array[:,11] <= 200) & # MET $p_T$ [GeV]
                        #(df_array[:,12] <= 200) & # MET $\phi$ 
                        #(df_array[:,13] <= 3) #& # '$H_T$ (had) [GeV]'
                        (df_array[:,14] <= 3) & # 'num_jets'
                        #(df_array[:,15] <= 200) & # $H_T$ (main) [GeV] 
                        #(df_array[:,16] <= 200) & # MET sig [GeV]
                        #(df_array[:,17] <= 200) & # $m_{eff}$ [GeV]
                        #(df_array[:,18] <= 200) & # $m_T$ photon [GeV]
                        #(df_array[:,19] <= 200) & # $m_T$ jet [GeV] 
                        #(df_array[:,20] <= 200) & # $m_T$ lepton [GeV]
                        #(df_array[:,21] <= 200) & # $\Delta \phi$ (MET-photon) 
                        #(df_array[:,22] >= 200) & # $\Delta \phi$ (MET-jet)
                        #(df_array[:,23] >= 10) & # $\Delta \phi$ (MET-lepton)
                        #(df_array[:,24] >= 0) & # $\Delta R$ (photon-jet)
                        #(df_array[:,25] <= 350) & #$\Delta R$ (photon-lepton)
                        #(df_array[:,26] <= 3)  # $\Delta R$ (lepton-jet) 
                        (df_array[:,10] <= 300)  # solo para no estar borrando los &
                      )[0]
len(indices_cut)

3739492

In [31]:
n_np = len(np.where(df_array[indices_cut,27] == 0)[0])

n_wgamma  = len(np.where(df_array[indices_cut,27] == 1)[0])
n_wjets   = len(np.where(df_array[indices_cut,27] == 2)[0])
n_ttgamma = len(np.where(df_array[indices_cut,27] == 3)[0])

n_sm = n_wgamma + n_wjets + n_ttgamma
print('Sobreviven ' + str(n_np) + ' del BP' + str(bp+1))

print('Sobreviven ' + str(n_wgamma) + ' de W gamma')
print('Sobreviven ' + str(n_wjets) + ' de W jets')
print('Sobreviven ' + str(n_ttgamma) + ' de tt gamma')
print('Sobreviven ' + str(n_sm) + ' de Background')

Sobreviven 204163 del BP1
Sobreviven 2964715 de W gamma
Sobreviven 18962 de W jets
Sobreviven 551652 de tt gamma
Sobreviven 3535329 de Background


In [14]:
ttgamma_cross = 0.144 * 1000 #[fb]
ttgamma_NLO   = 1.46 # k-factor [14 TeV]

Wgamma_cross = 0.944 * 1000 #[fb]
Wgamma_NLO   = 2.37 # k-factor [14 TeV]

Wjets_cross = 168.12 * 1000 #[fb]
Wjets_NLO   = 1.27 # k-factor [14 TeV]

In [15]:
ttgamma_aceptancia_init = n_ttgamma / (100000 * 150)
wgamma_aceptancia_init = n_wgamma / (100000 * 137) 
wjets_aceptancia_init = n_wjets / (100000 * 325 + 2000000)

NameError: name 'n_ttgamma' is not defined

In [19]:
luminosidad = 100

ttgamma_expected = ttgamma_cross * ttgamma_NLO * ttgamma_aceptancia_init * luminosidad
wgamma_expected  = Wgamma_cross * Wgamma_NLO * wgamma_aceptancia_init * luminosidad
wjets_expected   = Wjets_cross * Wjets_NLO * wjets_aceptancia_init * luminosidad

print('Esperamos ' + str(ttgamma_expected) + ' tt gamma')
print('Esperamos ' + str(wgamma_expected) + ' w gamma')
print('Esperamos ' + str(wjets_expected) + ' w jets')

Esperamos 2876.4293952 tt gamma
Esperamos 58446.31078773723 w gamma
Esperamos 160446.21376695653 w jets


In [35]:
cross_section_bps    = np.asarray([190, 189, 189, 92, 92, 92, 49, 48, 48, 27, 27, 27, 16, 16, 16])
branching_ratios_bps = np.asarray([0.14, 0.37, 0.73, 0.15, 0.36, 0.67, 0.16, 0.36, 0.62, 0.17, 0.35, 0.58, 0.16, 0.32, 0.52])


signal_init_sim_eve = np.asarray([836673,1e6,1.4e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6])
signal_cut_ev       = np.asarray([238160,246196,24249,301575,266047,203155,315960,280065,225958,326687,292051,244271,334782,301215,258277])

In [36]:
signal_cross        = cross_section_bps[bp] * branching_ratios_bps[bp] * 0.2134 # fb 
#El 0.2134 es suma de los partial widths del W a muon y a electron
signal_NLO          = 1.25

cross       = signal_cross * signal_NLO
aceptancia  = n_np / signal_init_sim_eve[bp]

signal_fidcross = cross * aceptancia
S_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', S_expected)

fiducial cross section:  1.7314396121901867
Events expected:  173.14396121901868


In [37]:
# Formula naive
S_expected / np.sqrt(ttgamma_expected + wgamma_expected + wjets_expected)

0.7014785854252812

In [38]:
# Formula discovery
B_expected = ttgamma_expected + wgamma_expected + wjets_expected
np.sqrt(2 * ((B_expected + S_expected) * np.log(1 + S_expected / B_expected) - S_expected))

0.7011467132183254

# Automatic cuts

In [15]:
bp = 1
df = pd.DataFrame()


lst = ['photon $p_T$ [GeV]', 'photon $\eta$', 'photon $\phi$', 'num photon',
       'jet $p_T$ [GeV]', 'jet $\eta$', 'jet $\phi$',
       'lepton $p_T$ [GeV]', 'lepton $\eta$', 'lepton $\phi$', 'num lepton',
       'MET $p_T$ [GeV]', 'MET $\phi$',
       '$H_T$ (had) [GeV]', 'num_jets',
       '$H_T$ (main) [GeV]', 'MET sig [GeV]', '$m_{eff}$ [GeV]',
       '$m_T$ photon [GeV]', '$m_T$ jet [GeV]', '$m_T$ lepton [GeV]',
       '$\Delta \phi$ (MET-photon)', '$\Delta \phi$ (MET-jet)', '$\Delta \phi$ (MET-lepton)', 
       '$\Delta R$ (photon-jet)', '$\Delta R$ (photon-lepton)', '$\Delta R$ (lepton-jet)', 
       'SM=0 - NP=1']

df[lst[0]]= np.concatenate([photon_SM[:,0],photon_NP[bp][:,0]])
df[lst[1]]= np.concatenate([photon_SM[:,1],photon_NP[bp][:,1]])
df[lst[2]]= np.concatenate([photon_SM[:,2],photon_NP[bp][:,2]])
df[lst[3]]= np.concatenate([photon_SM[:,3],photon_NP[bp][:,3]])

df[lst[4]]= np.concatenate([jet_SM[:,0],jet_NP[bp][:,0]])
df[lst[5]]= np.concatenate([jet_SM[:,1],jet_NP[bp][:,1]])
df[lst[6]]= np.concatenate([jet_SM[:,2],jet_NP[bp][:,2]])

df[lst[7]]= np.concatenate([lepton_SM[:,0],lepton_NP[bp][:,0]])
df[lst[8]]= np.concatenate([lepton_SM[:,1],lepton_NP[bp][:,1]])
df[lst[9]]= np.concatenate([lepton_SM[:,2],lepton_NP[bp][:,2]])
df[lst[10]]= np.concatenate([lepton_SM[:,3],lepton_NP[bp][:,3]])

df[lst[11]]= np.concatenate([MET_SM[:,0],MET_NP[bp][:,0]])
df[lst[12]]= np.concatenate([MET_SM[:,2],MET_NP[bp][:,2]])

df[lst[13]]= np.concatenate([hadinfo_SM[:,0],hadinfo_NP[bp][:,0]])
df[lst[14]]= np.concatenate([hadinfo_SM[:,1],hadinfo_NP[bp][:,1]])

df[lst[15]]= np.concatenate([HT_main_SM,HT_main_NP[bp][:]])
df[lst[16]]= np.concatenate([MET_sig_SM,MET_sig_NP[bp][:]])
df[lst[17]]= np.concatenate([m_eff_SM,m_eff_NP[bp][:]])

df[lst[18]]= np.concatenate([mT_photon_SM,mT_photon_NP[bp][:]])
df[lst[19]]= np.concatenate([mT_jet_SM,mT_jet_NP[bp][:]])
df[lst[20]]= np.concatenate([mT_lepton_SM,mT_lepton_NP[bp][:]])

df[lst[21]]= np.concatenate([phi_MET_photon_SM,phi_MET_photon_NP[bp][:]])
df[lst[22]]= np.concatenate([phi_MET_jet_SM,phi_MET_jet_NP[bp][:]])
df[lst[23]]= np.concatenate([phi_MET_lepton_SM,phi_MET_lepton_NP[bp][:]])

df[lst[24]]= np.concatenate([deltaR_photon_jet_SM,deltaR_photon_jet_NP[bp][:]])
df[lst[25]]= np.concatenate([deltaR_photon_lepton_SM,deltaR_photon_lepton_NP[bp][:]])
df[lst[26]]= np.concatenate([deltaR_lepton_jet_SM,deltaR_lepton_jet_NP[bp][:]])


df['bkg_flag']= np.concatenate([np.zeros(len(photon_NP[bp])),bkg_flag])

df[lst[27]]= np.concatenate([np.zeros(len(photon_SM)),np.ones(len(photon_NP[bp][:]))])

df_array = np.asarray(df)

In [16]:
cross_section_bps    = np.asarray([190, 189, 189, 92, 92, 92, 49, 48, 48, 27, 27, 27, 16, 16, 16])
branching_ratios_bps = np.asarray([0.14, 0.37, 0.73, 0.15, 0.36, 0.67, 0.16, 0.36, 0.62, 0.17, 0.35, 0.58, 0.16, 0.32, 0.52])


signal_init_sim_eve = np.asarray([836673,1e6,1.4e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6,1e6])
signal_cut_ev       = np.asarray([238160,246196,24249,301575,266047,203155,315960,280065,225958,326687,292051,244271,334782,301215,258277])

signal_cross        = cross_section_bps[bp] * branching_ratios_bps[bp] * 0.2134 # fb 
#El 0.2134 es suma de los partial widths del W a muon y a electron
signal_NLO          = 1.25

cross = signal_cross * signal_NLO

In [27]:
nbins = 10
indices_aux = np.arange(len(df_array))
sup_cuts = {}
inf_cuts = {}
random_vars = np.random.choice(range(27), size = 50, replace = True)

for ivar in random_vars:
    #print(df.columns[ivar])
    cuts = np.linspace(np.min(df_array[indices_aux,ivar]), np.max(df_array[indices_aux,ivar]), nbins)
    Z_sup = np.zeros(nbins)
    Z_inf = np.zeros(nbins)
    #%print(np.min(df_array[indices_aux,ivar]))
    #%print(np.max(df_array[indices_aux,ivar]))
    #print(len(indices_aux))

    for icut, cut in enumerate(cuts): 
        
        # Superior ------------------------------------------------------------------------------------
        indices_sup = indices_aux[np.where( (df_array[indices_aux,ivar] >= cut) )[0]]
        #if icut == 0: indices_sup = indices_aux
        
        n_np      = len(np.where(df_array[indices_sup, 27] == 0)[0])
        n_wgamma  = len(np.where(df_array[indices_sup, 27] == 1)[0])
        n_wjets   = len(np.where(df_array[indices_sup, 27] == 2)[0])
        n_ttgamma = len(np.where(df_array[indices_sup, 27] == 3)[0])

        ttgamma_expected = ttgamma_cross * ttgamma_NLO * (n_ttgamma / (100000 * 150)) * luminosidad
        wgamma_expected  = Wgamma_cross * Wgamma_NLO * (n_wgamma / (100000 * 137) ) * luminosidad
        wjets_expected   = Wjets_cross * Wjets_NLO * (n_wjets / (100000 * 325 + 2000000)) * luminosidad

        S_expected = cross * (n_np / signal_init_sim_eve[bp]) * luminosidad

        # Formula discovery
        B_expected = ttgamma_expected + wgamma_expected + wjets_expected
        if B_expected > 5: Z_sup[icut] = np.sqrt(2 * ((B_expected + S_expected) * np.log(1 + S_expected / B_expected) - S_expected))
        #%print('var >= ' + str(cut) + '=' + str(len(indices_sup)) + ' z = ' + str(Z_sup[icut]))
        #%print('np >=' + str(n_np))
        #%print('wgamma >=' + str(n_wgamma))
        #%print('ttgamma >=' + str(n_ttgamma))
        #%print('wjets >=' + str(n_wjets))

        #----------------------------------------------------------------------------------------------------

        # Inferior ------------------------------------------------------------------------------------
        indices_inf = indices_aux[np.where( (df_array[indices_aux,ivar] <= cut) )[0]]
        if icut == (nbins - 1): indices_inf = indices_aux
        n_np      = len(np.where(df_array[indices_inf, 27] == 0)[0])
        n_wgamma  = len(np.where(df_array[indices_inf, 27] == 1)[0])
        n_wjets   = len(np.where(df_array[indices_inf, 27] == 2)[0])
        n_ttgamma = len(np.where(df_array[indices_inf, 27] == 3)[0])

        ttgamma_expected = ttgamma_cross * ttgamma_NLO * (n_ttgamma / (100000 * 150)) * luminosidad
        wgamma_expected  = Wgamma_cross * Wgamma_NLO * (n_wgamma / (100000 * 137) ) * luminosidad
        wjets_expected   = Wjets_cross * Wjets_NLO * (n_wjets / (100000 * 325 + 2000000)) * luminosidad

        S_expected = cross * (n_np / signal_init_sim_eve[bp]) * luminosidad

        # Formula discovery
        B_expected = ttgamma_expected + wgamma_expected + wjets_expected
        if B_expected > 5: Z_inf[icut] = np.sqrt(2 * ((B_expected + S_expected) * np.log(1 + S_expected / B_expected) - S_expected))
        #%print('var <= ' + str(cut) + '=' + str(len(indices_inf)) + ' z = ' + str(Z_inf[icut]))
        #%print('np >=' + str(n_np))
        #%print('wgamma >=' + str(n_wgamma))
        #%print('ttgamma >=' + str(n_ttgamma))
        #%print('wjets >=' + str(n_wjets))

        #----------------------------------------------------------------------------------------------------

    max_cut = np.argmax((Z_inf, Z_sup))
    #%print(Z_inf)
    #%print(Z_sup)
    if max_cut >= len(Z_inf):
        print(df.columns[ivar] + ' >= ' + str(cuts[max_cut - len(Z_sup)]) + '; Z = ' + str(Z_sup[max_cut - len(Z_inf)]))
        indices_aux = indices_aux[np.where( (df_array[indices_aux,ivar] >= cuts[max_cut - len(Z_sup)]) )[0]]
        if df.columns[ivar] in inf_cuts.keys():
            inf_cuts[df.columns[ivar]] = cuts[max_cut - len(Z_sup)]
        inf_cuts.setdefault(df.columns[ivar], cuts[max_cut - len(Z_sup)])
        sup_cuts.setdefault(df.columns[ivar], 999999)
    else:
        print(df.columns[ivar] + ' <= ' + str(cuts[max_cut]) + '; Z = ' + str(Z_inf[max_cut]))
        indices_aux = indices_aux[np.where( (df_array[indices_aux,ivar] <= cuts[max_cut]) )[0]]
        if df.columns[ivar] in sup_cuts.keys():
            sup_cuts[df.columns[ivar]] = cuts[max_cut]
        sup_cuts.setdefault(df.columns[ivar], cuts[max_cut])
        inf_cuts.setdefault(df.columns[ivar], -999999)
    
    #print(len(indices_aux))       
    
    
    
    
n_np = len(np.where(df_array[indices_aux, 27] == 0)[0])

n_wgamma  = len(np.where(df_array[indices_aux, 27] == 1)[0])
n_wjets   = len(np.where(df_array[indices_aux, 27] == 2)[0])
n_ttgamma = len(np.where(df_array[indices_aux, 27] == 3)[0])

ttgamma_expected = ttgamma_cross * ttgamma_NLO * (n_ttgamma / (100000 * 150)) * luminosidad
wgamma_expected  = Wgamma_cross * Wgamma_NLO * (n_wgamma / (100000 * 137) ) * luminosidad
wjets_expected   = Wjets_cross * Wjets_NLO * (n_wjets / (100000 * 325 + 2000000)) * luminosidad

S_expected = cross * (n_np / signal_init_sim_eve[bp]) * luminosidad

# Formula discovery
B_expected = ttgamma_expected + wgamma_expected + wjets_expected
if B_expected > 5: 
    print('\n')
    print('Z = ', np.sqrt(2 * ((B_expected + S_expected) * np.log(1 + S_expected / B_expected) - S_expected)))
    print('Expected Background = ' + str(B_expected))
    print('Expected Signal = ' + str(S_expected))

$H_T$ (main) [GeV] <= 4507.365555555555; Z = 0.9747136617796793
MET $\phi$ <= 3.142; Z = 0.9747136617796793
photon $\phi$ <= 3.142; Z = 0.9747136617796793
$\Delta R$ (lepton-jet) <= 4.17858158245534; Z = 0.97611940116879
$H_T$ (main) [GeV] <= 4490.290000000001; Z = 0.97611940116879
jet $p_T$ [GeV] <= 681.0722222222222; Z = 0.9772421207731179
$\Delta \phi$ (MET-lepton) <= 6.279999999999999; Z = 0.9772421207731179
num_jets <= 2.666666666666667; Z = 1.3715864809309641
$\Delta R$ (photon-jet) <= 6.109305121582637; Z = 1.371599667346906
jet $\eta$ <= 4.447; Z = 1.371599667346906
$m_T$ photon [GeV] <= 1780.3297131593195; Z = 1.37160144853898
MET $p_T$ [GeV] <= 936.72; Z = 1.371602809145103
MET $\phi$ <= 3.142; Z = 1.371602809145103
jet $\eta$ <= 4.447; Z = 1.371602809145103
num photon <= 2.0; Z = 1.3716255431988682
MET $p_T$ [GeV] <= 916.6; Z = 1.3716255431988682
lepton $p_T$ [GeV] <= 668.1688888888889; Z = 1.3716546441712003
$H_T$ (main) [GeV] <= 1464.6666666666667; Z = 1.371704623189818
le

In [28]:
inf_cuts

{'$H_T$ (main) [GeV]': -999999,
 'MET $\\phi$': -999999,
 'photon $\\phi$': -999999,
 '$\\Delta R$ (lepton-jet)': -999999,
 'jet $p_T$ [GeV]': -999999,
 '$\\Delta \\phi$ (MET-lepton)': -999999,
 'num_jets': -999999,
 '$\\Delta R$ (photon-jet)': -999999,
 'jet $\\eta$': -999999,
 '$m_T$ photon [GeV]': -999999,
 'MET $p_T$ [GeV]': -999999,
 'num photon': -999999,
 'lepton $p_T$ [GeV]': -999999,
 'lepton $\\eta$': -999999,
 'photon $\\eta$': -999999,
 'lepton $\\phi$': -999999,
 '$m_T$ lepton [GeV]': -999999,
 '$\\Delta \\phi$ (MET-photon)': -999999,
 '$H_T$ (had) [GeV]': -999999,
 'jet $\\phi$': -999999,
 'num lepton': -999999}

In [29]:
sup_cuts

{'$H_T$ (main) [GeV]': 1464.32,
 'MET $\\phi$': 3.142,
 'photon $\\phi$': 3.142,
 '$\\Delta R$ (lepton-jet)': 4.17858158245534,
 'jet $p_T$ [GeV]': 641.42,
 '$\\Delta \\phi$ (MET-lepton)': 6.279999999999999,
 'num_jets': 2.0,
 '$\\Delta R$ (photon-jet)': 6.10142544048846,
 'jet $\\eta$': 4.447,
 '$m_T$ photon [GeV]': 1742.5474235202296,
 'MET $p_T$ [GeV]': 916.6,
 'num photon': 2.0,
 'lepton $p_T$ [GeV]': 593.82,
 'lepton $\\eta$': 2.5,
 'photon $\\eta$': 2.369,
 'lepton $\\phi$': 3.142,
 '$m_T$ lepton [GeV]': 113.90635877950581,
 '$\\Delta \\phi$ (MET-photon)': 6.281,
 '$H_T$ (had) [GeV]': 641.42,
 'jet $\\phi$': 3.142,
 'num lepton': 1.0}

In [48]:
np.where(df.columns == variables[i])[0][0]

15

In [49]:
indices_aux = np.arange(len(df_array))

variables = list(sup_cuts.keys())
for i in range(len(variables)):
    ind_var = np.where(df.columns == variables[i])[0][0]
    indices_aux = indices_aux[np.where( (df_array[indices_aux, ind_var] <= sup_cuts[variables[i]]) & 
                            (df_array[indices_aux, ind_var] > inf_cuts[variables[i]])
                           )[0]]


In [50]:
len(df_array)

6138425

In [51]:
len(indices_aux)

2787978

In [52]:
n_np = len(np.where(df_array[indices_aux, 27] == 0)[0])

n_wgamma  = len(np.where(df_array[indices_aux, 27] == 1)[0])
n_wjets   = len(np.where(df_array[indices_aux, 27] == 2)[0])
n_ttgamma = len(np.where(df_array[indices_aux, 27] == 3)[0])

ttgamma_expected = ttgamma_cross * ttgamma_NLO * (n_ttgamma / (100000 * 150)) * luminosidad
wgamma_expected  = Wgamma_cross * Wgamma_NLO * (n_wgamma / (100000 * 137) ) * luminosidad
wjets_expected   = Wjets_cross * Wjets_NLO * (n_wjets / (100000 * 325 + 2000000)) * luminosidad

S_expected = cross * (n_np / signal_init_sim_eve[bp]) * luminosidad

# Formula discovery
B_expected = ttgamma_expected + wgamma_expected + wjets_expected
if B_expected > 5: 
    print(np.sqrt(2 * ((B_expected + S_expected) * np.log(1 + S_expected / B_expected) - S_expected)))


1.3909840737247157


In [53]:
S_expected

310.780227681

In [54]:
B_expected

49815.119655262206

In [55]:
n_np

166604

In [56]:
signal_init_sim_eve[bp]

1000000.0

In [57]:
cross

18.6538275

In [58]:
luminosidad

100